In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from pprint import pprint

In [ ]:
#Reading data from CSV file using pandas dataframe. We have dropped the 7th feature (Var7) from the
#data set as it created some unexpected error 

df = pd.read_csv("variance.csv")
df = df.drop("File Name", axis=1)
df = df.drop("Var7", axis=1)

In [ ]:
df = df.rename(columns = {"CLASSLABEL":"label"})

In [ ]:
#This function is used for splitting the data into training set (train_df), validation set (val_df) and testing set (test_df)

def Data_split(df,test_size, val_size):
    if isinstance(test_size,float):
        test_size = round(test_size*len(df))
        val_size = round(val_size*len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)        #Random sampling of the data set
    test_df = df.loc[test_indices]
    rest_df = df.drop(test_indices)
    
    indices1 = rest_df.index.tolist()
    val_indices = random.sample(population=indices1, k=val_size)         # Random sampling of the remainder of the data set
    val_df = rest_df.loc[val_indices]
    train_df = rest_df.drop(val_indices)
    
    return train_df, test_df, val_df

In [ ]:
random.seed(0)
train_df, test_df, val_df = Data_split(df,test_size=20, val_size = 30)  #Partitioning the training, validation and testing data

In [ ]:
data=train_df.values

In [ ]:
# This function is used to check the purity of node, i.e whether there is one class label present at a node or if more than 
# one (in our case two class labels, 1 and 0) class labels are present in a particular node.

def check_node_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [ ]:
# This function is used to check how many classes are there in the dataset
# the last column of the dataset contains the class-label
# In our case, label 1 corresponds to motor image stimulation and label 0 corresponds to visual stimulation
def classify_data(data):

    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column,return_counts=True)
    
    index = counts_unique_classes.argmax()
    classification = unique_classes[index]

    return classification

In [ ]:
# This function is used to find the mid values of two consecutive values of a particular feature and to consider that
# as a potential value to form a question in a node in the tree

def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape 
    for column_index in range(n_columns-1):
        potential_splits[column_index] = []
        values = data[:,column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index-1]
                potential_split = (current_value+previous_value)/2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [ ]:
# This function is used to divide the dataset at a node based on a particular yes/no question
# The questions are asked in the following form 'Is the feature < a particular split value?'
# data_below stores the data that has the considered feature with values less or equal to the split value
#data_above stores the data that has the considered feature with values greater than the split value

def data_partition_by_question(data, split_column, split_value):
    
    split_column_values = data[:,split_column]
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    
    return data_below, data_above

In [ ]:
# This function is used to compute entropy based on the frequency of occurence of the class labels at a node

def entropy_calculation(data):
    
    label_column = data[:,-1]
    _, counts = np.unique(label_column, return_counts=True)
    
    probabilities = counts/counts.sum()
    entropy = sum(probabilities*-np.log2(probabilities))
       
    return entropy

In [ ]:
# This function is used to find the entropy gain at a node

def calculate_overall_entropy(data_below, data_above):
    
    n_data_points = len(data_below)+len(data_above)      #Calculating total number of data points

    p_data_below = len(data_below)/n_data_points      #Calculating frequency (probability)
    p_data_above = len(data_above)/n_data_points      #Calculating frequency (probability) 

    overall_entropy = (p_data_below*entropy_calculation(data_below))+(p_data_above*entropy_calculation(data_above)) 
    
    return overall_entropy

In [ ]:
# This function is used to select the best feature and the corresponding value to form the yes/no questionto be asked
# at a node on the basis of enropy gain

def determine_best_split(data, potential_splits):
    overall_entropy = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = data_partition_by_question(data,split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
        
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [ ]:
# This is the main decision tree algorithm. If all the examples at a node are of same label then that class-label is returned.
# Otherwise, if there is impurity at a node then this function is called recursively to grow the decision tree


def decision_tree_algorithm(df, counter=0):
    if counter == 0:
        global column_headers             #This part is used to change the data from pandas dataframe to numpy 2D array
        column_headers = df.columns
        data = df.values
    else:
        data = df
        
    
    if check_node_purity(data):
        classification = classify_data(data)        #calss label is returned for nodes, where all the data are of same class
        return classification                       # i.e. the node is not impure and the node is declared as a leaf
        
    
        #recursive part for impure nodes
        
    else:
        counter+=1
            
        #calling other functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data,potential_splits)
        data_below, data_above = data_partition_by_question(data, split_column, split_value)
            
        # sub-tree create
        feature_name = column_headers[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
            
        yes_answer = decision_tree_algorithm(data_below,counter)
        no_answer = decision_tree_algorithm(data_above,counter)
            
        if yes_answer ==no_answer :
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
        

In [ ]:
tree = decision_tree_algorithm(train_df)      # training the decision tree based on the training data set (train_df)

In [ ]:
# This function is used to get a class label for a new data from the trained decision tree structure

def classification(example, tree):
    question = list(tree.keys())[0]
    feature_name, operator, value = question.split()

    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]
    
    if not isinstance(answer,dict):
        return answer
    else:
        residual_tree = answer
        return classification(example, residual_tree)

In [ ]:
# This function calls the 'classification' function on the entire testing set and there by checks the accuracy of the algorithm
# Once all the labels are determined from the algorithm we count for how many cases the class labels provided by the algorithm
# and the class labels provided by the data set (testing data) are equal. We count such equal examples. If for 'm' cases the two
# sets of class labels are same and there are n number of data points in testing set, then accuracy = m/n, ie accuracy is the
# mean value of the true counts

def calculate_accuracy(df, tree):
    
    
    df["classification"] = df.apply(classification, axis = 1, args=(tree,))
    df["classification_correct"] = df.classification == df.label
    
    accuracy = df.classification_correct.mean()

    return accuracy

In [ ]:
# Accuracy obtained on the the testing set of the maximum depth decision tree (Overfitted Case)
calculate_accuracy(test_df, tree)

0.55

In [ ]:
# This function is used to locate the appropriate feature values based on which a particular question is asked at a node

def select_feature_df(df, question):
    
    feature, _, value = question.split()
    
    df_yes = df[df[feature]<=float(value)]
    df_no = df[df[feature]>float(value)]
    
    return df_yes, df_no

In [ ]:
# This function is used for pruning a particular node

def pruning_result(tree, train_df, val_df):
    
    leaf = train_df.label.value_counts().index[0]
    errors_leaf = sum(val_df.label != leaf)

    val_df["classification"] = val_df.apply(classification, axis = 1, args=(tree,))
    make_prediction = val_df["classification"]                                  # Checking the output of the validation dataset

    errors_decision_node = sum(val_df.label!=make_prediction)

    if errors_leaf <= errors_decision_node:
        return leaf
    else:
        return tree
    

In [ ]:
# This function is used for pruning the entire decision tree. Pruning is performed as long as the error in the validation set is less than the error in the training set. We consider a node and make it a lef by determining the class by majority in  that node. We compute the accuracy in the validation set and compare it with error obtained by the non pruned tree. If erroris reduced, pruning is performed, else not. Pruning is performed recursively.

def post_pruning(tree, train_df, val_df):
    
    question = list(tree.keys())[0]
    yes_answer, no_answer = tree[question]

    #basecase
    if not isinstance(yes_answer,dict) and not isinstance(no_answer,dict):
        return pruning_result(tree, train_df, val_df)
        
        
        #recursion
    else:
        
        train_df_yes, train_df_no = select_feature_df(train_df, question)  
        val_df_yes, val_df_no = select_feature_df(val_df, question)
        
        if isinstance(yes_answer, dict):
            tree[question][0] = post_pruning(yes_answer, train_df_yes, val_df_yes)
        
        if isinstance(no_answer, dict):
            tree[question][1] = post_pruning(no_answer, train_df_no, val_df_no)
        
        return pruning_result(tree, train_df, val_df)
    

In [ ]:
post_pruning(tree, train_df, val_df)    # Pruning the decision tree

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

{'Var1 <= 5009.133': [0,
  {'Var14 <= 28278.936999999998': [0.0,
    {'Var14 <= 38198.795999999995': [1.0, 0.0]}]}]}

In [ ]:
# Checking the accuracy on the testing set of the pruned tree based on a validation set

calculate_accuracy(test_df, tree)

0.85